In [2]:
!pip install diffusers transformers torch torchvision torchaudio accelerate pillow ultralytics streamlit -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 113.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 89.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 949.8/949.8 kB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import os
import random
import torch
from diffusers import StableDiffusionPipeline
from PIL import Image, ImageDraw
import yaml
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model_id = "CompVis/stable-diffusion-v1-4"  # Stable Diffusion v1.4
pipe = StableDiffusionPipeline.from_pretrained(model_id).to(device)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/492M [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [5]:
minerals = ["gold deposits", "manganese nodules", "cobalt-rich crusts", "rare earth minerals", "deep-sea sulfides"]
sea_animals = ["jellyfish", "giant squid", "anglerfish", "deep-sea crabs", "bioluminescent fish"]
seabed_features = ["hydrothermal vents", "coral formations", "underwater volcanoes", "ocean trenches", "rocky seabed"]

In [6]:
def generate_synthetic_dataset(num_images=100, save_folder="underwater_dataset"):
    img_folder = os.path.join(save_folder, "images")
    labels_folder = os.path.join(save_folder, "labels")

    os.makedirs(img_folder, exist_ok=True)
    os.makedirs(labels_folder, exist_ok=True)

    for i in range(num_images):
        # Generate a dynamic prompt
        mineral = random.choice(minerals)
        sea_animal = random.choice(sea_animals)
        seabed = random.choice(seabed_features)
        prompt = f"Underwater deep-sea mining, {mineral}, {sea_animal} swimming, {seabed}, realistic, high-quality"

        # Generate image
        image = pipe(prompt).images[0]
        image_path = f"{img_folder}/underwater_{i+1}.jpg"
        image.save(image_path)

        # Generate dummy YOLO annotations (center_x, center_y, width, height)
        width, height = image.size
        obj_x, obj_y = random.uniform(0.2, 0.8), random.uniform(0.2, 0.8)
        obj_w, obj_h = random.uniform(0.1, 0.3), random.uniform(0.1, 0.3)

        label_path = f"{labels_folder}/underwater_{i+1}.txt"
        with open(label_path, "w") as f:
            f.write(f"0 {obj_x} {obj_y} {obj_w} {obj_h}\n")  # Class 0 (deep-sea mineral)

        print(f"✅ Generated Image {i+1}/{num_images} with prompt: {prompt}")

    print("🎉 Successfully generated dataset!")

In [7]:
generate_synthetic_dataset(100)


  0%|          | 0/50 [00:00<?, ?it/s]

✅ Generated Image 1/100 with prompt: Underwater deep-sea mining, cobalt-rich crusts, deep-sea crabs swimming, hydrothermal vents, realistic, high-quality


  0%|          | 0/50 [00:00<?, ?it/s]

✅ Generated Image 2/100 with prompt: Underwater deep-sea mining, deep-sea sulfides, bioluminescent fish swimming, underwater volcanoes, realistic, high-quality


  0%|          | 0/50 [00:00<?, ?it/s]

...

✅ Generated Image 98/100 with prompt: Underwater deep-sea mining, manganese nodules, anglerfish swimming, rocky seabed, realistic, high-quality


  0%|          | 0/50 [00:00<?, ?it/s]

✅ Generated Image 99/100 with prompt: Underwater deep-sea mining, cobalt-rich crusts, anglerfish swimming, underwater volcanoes, realistic, high-quality


  0%|          | 0/50 [00:00<?, ?it/s]

✅ Generated Image 100/100 with prompt: Underwater deep-sea mining, gold deposits, bioluminescent fish swimming, underwater volcanoes, realistic, high-quality
🎉 Successfully generated dataset!


In [10]:
import shutil
from google.colab import files

shutil.make_archive("underwater_images", 'zip', "underwater_dataset/images")
files.download("underwater_images.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [21]:
yaml_content = """
path: /content/underwater_dataset
train: images  # Folder with training images
val: images    # Using the same images for validation (modify if you have separate val set)
test: images   # Using the same images for testing (modify if needed)

nc: 1  # Number of classes (change if needed)
names: ['deep_sea_mineral']  # Modify according to your dataset
"""

# Save the YAML file
with open("/content/underwater_dataset.yaml", "w") as f:
    f.write(yaml_content)

print("✅ YAML file saved correctly!")


✅ YAML file saved correctly!


In [22]:
import os

dataset_path = "/content/underwater_dataset"

print("Dataset directory exists:", os.path.exists(dataset_path))
print("Images folder exists:", os.path.exists(os.path.join(dataset_path, "images")))
print("Labels folder exists:", os.path.exists(os.path.join(dataset_path, "labels")))

# Check the number of images and labels
image_files = os.listdir(os.path.join(dataset_path, "images")) if os.path.exists(os.path.join(dataset_path, "images")) else []
label_files = os.listdir(os.path.join(dataset_path, "labels")) if os.path.exists(os.path.join(dataset_path, "labels")) else []

print(f"Total Images Found: {len(image_files)}")
print(f"Total Labels Found: {len(label_files)}")


Dataset directory exists: True
Images folder exists: True
Labels folder exists: True
Total Images Found: 100
Total Labels Found: 100


In [23]:
from ultralytics import YOLO

# Load the YOLOv8 model (nano version)
model = YOLO("yolov8n.yaml")  # You can also try 'yolov8s.yaml' for a stronger model

# Train the model
model.train(
    data="/content/underwater_dataset.yaml",  # Path to dataset configuration
    epochs=10,        # Number of epochs
    imgsz=640,        # Image size
    batch=16,         # Batch size
    device="cuda"     # Use GPU for faster training
)


Ultralytics 8.3.94 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.yaml, data=/content/underwater_dataset.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cuda, workers=8, project=None, name=train7, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, s

100%|██████████| 755k/755k [00:00<00:00, 139MB/s]

Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           


  7                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  8                  -1  1    460288  ultralytics.nn.modules.block.C2f             [256, 256, 1, True]           
  9                  -1  1    164608  ultralytics.nn.modules.block.SPPF            [256, 256, 5]                 
 10                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None, 2, 'nearest']          
 11             [-1, 6]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 12                  -1  1    148224  ultralytics.nn.modules.block.C2f             [384, 128, 1]                 
 13                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None, 2, 'nearest']          
 14             [-1, 4]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 15                  -1  1     37248  ultralytics.nn.modules.block.C2f             [192,

100%|██████████| 5.35M/5.35M [00:00<00:00, 291MB/s]


...

                   all        100        100   0.000367       0.11   0.000212   3.92e-05



10 epochs completed in 0.011 hours.
Optimizer stripped from runs/detect/train7/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train7/weights/best.pt, 6.2MB

Validating runs/detect/train7/weights/best.pt...
Ultralytics 8.3.94 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLOv8n summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:02<00:00,  1.52it/s]


                   all        100        100   0.000367       0.11   0.000212   3.92e-05
Speed: 0.4ms preprocess, 3.2ms inference, 0.0ms loss, 10.3ms postprocess per image
Results saved to runs/detect/train7


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7b1b24b97a50>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 